In [15]:
import pandas as pd
import numpy as np
from scipy import spatial
import polars as pl
import random
import os

In [2]:
weapons = ["bomb", "rifle", "arrow"]

In [3]:
arr_random_input = np.random.default_rng().uniform(low=-100,high=100,size=[1000000,2])
arr_random_query = np.random.default_rng().uniform(low=-100,high=100,size=[1000,2])
df = pd.DataFrame(arr_random_input, columns=["x","y"])
query_df = pd.DataFrame(arr_random_query, columns=["x","y"])
df["weapon"] = "a"
df["weapon"]=df.weapon.apply(lambda x:random.choice(weapons))
df.to_parquet("notebook_results/input.parquet")
query_df.to_parquet("notebook_results/query.parquet")

### This seems to work, have to create the algorithm for it

In [5]:
# This is 20x better than the original
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)


In [6]:
%%timeit
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)

379 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


try with polars

In [7]:
query_df_p = pl.read_parquet("notebook_results/query.parquet")
df_p = pl.read_parquet("notebook_results/input.parquet")

In [8]:
#df_p = df_p.with_row_index("rownum",0)

In [9]:
tree = spatial.KDTree(df_p.select("x","y"))

In [10]:
tree.query(query_df_p[1])[1]

array([387074], dtype=int64)

In [11]:
for row in query_df_p.iter_rows():
    asd = df_p[int(tree.query(row)[1])]["weapon"]

In [12]:
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], "weapon": tree.query(row)[1]})

In [13]:
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

In [14]:
%%timeit
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

338 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Cleanup trial

In [16]:
os.listdir()

['compute',
 'input.parquet',
 'input.pickle',
 'notebook_results',
 'out.csv',
 'preproc',
 'preproc.py',
 'requirements.txt',
 'setup',
 'src.py',
 'trial.ipynb']

In [17]:
filelist = [ f for f in os.listdir() if f.endswith(".parquet") or f.endswith(".pickle")]

In [18]:
filelist

['input.parquet', 'input.pickle']

In [20]:
filelist = [ f for f in os.listdir() if f.endswith(".parquet") or 
            f.endswith(".pickle")]
for f in filelist:
    os.remove(f)